<a href="https://colab.research.google.com/github/SPThole/LTFS_repo/blob/main/LETSEE_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install xgboost==1.3.3
# !python -m pip install dask[dataframe] --upgrade
# !pip install dask-xgboost


# !pip install dask_ml
# !pip install shap

In [ ]:
!pip install catboost

In [ ]:
# from dask.distributed import Client

# client = Client(n_workers=4, threads_per_worker=1)
# client

In [ ]:
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import lightgbm
import catboost as catb
from sklearn.model_selection import GridSearchCV, StratifiedKFold
%matplotlib inline

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/pro_train_ltfs1.csv')
test = pd.read_csv('/content/drive/MyDrive/pro_test_ltfs1.csv')

In [ ]:
train = train.sort_values(by=['ID','REPORTED DATE - HIST'])
test = test.sort_values(by=['ID','REPORTED DATE - HIST'])

In [ ]:
train.head()

,ID,EMI,REPORTED DATE - HIST,CUR BAL - HIST,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,ASSET_CLASS,SELF-INDICATOR,TENURE,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,DisbursalDate,MaturityDAte,AuthDate,ManufacturerID,LTV,SEX,AGE,MonthlyIncome,State,Top-up Month,DURATION,lens
101,1,24000.0,2013-03-31 00:00:00,226605.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191
100,1,24000.0,2013-04-30 00:00:00,205875.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191
99,1,24000.0,2013-05-31 00:00:00,208846.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191
98,1,24000.0,2013-06-30 00:00:00,211860.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191
97,1,24000.0,2013-07-31 00:00:00,190917.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191


In [ ]:
train['how_many_6'] = (pd.to_datetime(train['REPORTED DATE - HIST'],errors='coerce')- pd.to_datetime(train['DisbursalDate'],errors='coerce'))/np.timedelta64(6, 'M')
test['how_many_6'] = (pd.to_datetime(test['REPORTED DATE - HIST'],errors='coerce')- pd.to_datetime(test['DisbursalDate'],errors='coerce'))/np.timedelta64(6, 'M')

In [ ]:
train['how_many_6'] = np.round(train['how_many_6'])
test['how_many_6'] = np.round(test['how_many_6'])

In [ ]:
traincur = train.groupby(['ID','how_many_6']).agg({'CUR BAL - HIST':'mean'}).reset_index()
testcur = test.groupby(['ID','how_many_6']).agg({'CUR BAL - HIST':'mean'}).reset_index()

In [ ]:
traincur = traincur[(traincur['how_many_6']<=30) & (traincur['how_many_6']>=1)]
testcur = testcur[(testcur['how_many_6']<=30) & (testcur['how_many_6']>=1)]

In [ ]:
testcur = testcur.pivot_table(values='CUR BAL - HIST', index='ID', columns='how_many_6').reset_index()
traincur = traincur.pivot_table(values='CUR BAL - HIST', index='ID', columns='how_many_6').reset_index()

In [ ]:
train.columns

Index(['ID', 'EMI', 'REPORTED DATE - HIST', 'CUR BAL - HIST', 'ACCT-TYPE',
       'CONTRIBUTOR-TYPE', 'OWNERSHIP-IND', 'ACCOUNT-STATUS', 'ASSET_CLASS',
       'SELF-INDICATOR', 'TENURE', 'Frequency', 'InstlmentMode', 'LoanStatus',
       'PaymentMode', 'BranchID', 'Tenure', 'AssetCost', 'AmountFinance',
       'DisbursalAmount', 'DisbursalDate', 'MaturityDAte', 'AuthDate',
       'ManufacturerID', 'LTV', 'SEX', 'AGE', 'MonthlyIncome', 'State',
       'Top-up Month', 'DURATION', 'lens', 'how_many_6'],
      dtype='object')

In [ ]:
for col in ['REPORTED DATE - HIST','DisbursalDate','MaturityDAte']:
  print(col)
  train[col+'m'] = train[col].apply(lambda x: str(x)[5:7])
  test[col+'m'] = test[col].apply(lambda x: str(x)[5:7])

  train[col+'y'] = train[col].apply(lambda x: str(x)[:4])
  test[col+'y'] = test[col].apply(lambda x: str(x)[:4])

for col in ['REPORTED DATE - HIST','DisbursalDate','MaturityDAte']:
  print(col)
  train[col+'m'] = pd.to_numeric(train[col+'m'],downcast='float',errors='coerce')
  test[col+'m'] = pd.to_numeric(test[col+'m'],downcast='float',errors='coerce')

  train[col+'y'] = pd.to_numeric(train[col+'y'], downcast='float',errors='coerce')
  test[col+'y'] = pd.to_numeric(test[col+'y'], downcast='float',errors='coerce')

REPORTED DATE - HIST
DisbursalDate
MaturityDAte
REPORTED DATE - HIST
DisbursalDate
MaturityDAte


In [ ]:
train.columns

Index(['ID', 'EMI', 'REPORTED DATE - HIST', 'CUR BAL - HIST', 'ACCT-TYPE',
       'CONTRIBUTOR-TYPE', 'OWNERSHIP-IND', 'ACCOUNT-STATUS', 'ASSET_CLASS',
       'SELF-INDICATOR', 'TENURE', 'Frequency', 'InstlmentMode', 'LoanStatus',
       'PaymentMode', 'BranchID', 'Tenure', 'AssetCost', 'AmountFinance',
       'DisbursalAmount', 'DisbursalDate', 'MaturityDAte', 'AuthDate',
       'ManufacturerID', 'LTV', 'SEX', 'AGE', 'MonthlyIncome', 'State',
       'Top-up Month', 'DURATION', 'lens', 'how_many_6',
       'REPORTED DATE - HISTm', 'REPORTED DATE - HISTy', 'DisbursalDatem',
       'DisbursalDatey', 'MaturityDAtem', 'MaturityDAtey'],
      dtype='object')

In [ ]:
train.head()

,ID,EMI,REPORTED DATE - HIST,CUR BAL - HIST,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,ASSET_CLASS,SELF-INDICATOR,TENURE,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,DisbursalDate,MaturityDAte,AuthDate,ManufacturerID,LTV,SEX,AGE,MonthlyIncome,State,Top-up Month,DURATION,lens,how_many_6,REPORTED DATE - HISTm,REPORTED DATE - HISTy,DisbursalDatem,DisbursalDatey,MaturityDAtem,MaturityDAtey
101,1,24000.0,2013-03-31 00:00:00,226605.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191,5.0,3.0,2013.0,2.0,2012.0,1.0,2016.0
100,1,24000.0,2013-04-30 00:00:00,205875.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191,5.0,4.0,2013.0,2.0,2012.0,1.0,2016.0
99,1,24000.0,2013-05-31 00:00:00,208846.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191,5.0,5.0,2013.0,2.0,2012.0,1.0,2016.0
98,1,24000.0,2013-06-30 00:00:00,211860.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191,6.0,6.0,2013.0,2.0,2012.0,1.0,2016.0
97,1,24000.0,2013-07-31 00:00:00,190917.0,1.28679,1.420227,1.458272,1.417701,1.427534,5,60.0,2,1,1,0.664989,0.700483,48,450000,275000.0,275000.0,2012-02-10,2016-01-15,2012-02-10,0.679963,61.11,1,49.0,35833.33,0.874595,6,1435 days,191,6.0,7.0,2013.0,2.0,2012.0,1.0,2016.0


In [ ]:
train = train.groupby('ID').agg({
   'EMI':'mean', 
  'ACCT-TYPE':'mean',
  'CONTRIBUTOR-TYPE':'mean', 
  'OWNERSHIP-IND':'mean', 
  'ACCOUNT-STATUS':'mean', 
  'ASSET_CLASS':'mean',
  'SELF-INDICATOR':'mean', 
  'TENURE':'mean', 
  'Frequency':'mean', 
  'InstlmentMode':'mean', 
  'LoanStatus':'sum',
       'PaymentMode':'mean', 
       'BranchID':'mean', 
       'Tenure':'mean', 
       'AssetCost':'mean', 
       'AmountFinance':'mean',
       'DisbursalAmount':'mean', 
       'ManufacturerID':'mean', 
       'LTV':'mean', 
       'SEX':'mean', 
       'AGE':'mean', 
       'MonthlyIncome':'mean', 
       'State':'mean',
       'Top-up Month':'mean', 
       'DURATION':'unique', 
       'lens':'mean', 
       'how_many_6':'mean',
       'REPORTED DATE - HISTm':'mean', 
       'REPORTED DATE - HISTy':'mean', 
       'DisbursalDatem':'mean',
       'DisbursalDatey':'mean', 
       'MaturityDAtem':'mean', 
       'MaturityDAtey':'mean',
    # 'DisbursalDatem':'mean',
    # 'DisbursalDatey':'mean',
    # 'MaturityDAtem':'mean',
    # 'MaturityDAtey':'mean',
    # 'EMI':'mean',
    # 'SELF-INDICATOR':'mean',
    # 'TENURE':'mean',
    # 'Frequency':'mean',
    # 'InstlmentMode':'mean',
    # 'LoanStatus':'sum',
    # 'PaymentMode':'mean',
    # 'Tenure':'mean',
    # 'AssetCost':'mean',
    # 'AmountFinance':'mean',
    # 'DisbursalAmount':'mean',
    # 'LTV':'mean','SEX':'mean',
    # 'AGE':'mean','MonthlyIncome':'mean',
    # 'Top-up Month':'unique','DURATION':'unique',
    # 'lens':'mean'

}).reset_index()

test = test.groupby('ID').agg({
    
    'EMI':'mean', 
  'ACCT-TYPE':'mean',
  'CONTRIBUTOR-TYPE':'mean', 
  'OWNERSHIP-IND':'mean', 
  'ACCOUNT-STATUS':'mean', 
  'ASSET_CLASS':'mean',
  'SELF-INDICATOR':'mean', 
  'TENURE':'mean', 
  'Frequency':'mean', 
  'InstlmentMode':'mean', 
  'LoanStatus':'sum',
       'PaymentMode':'mean', 
       'BranchID':'mean', 
       'Tenure':'mean', 
       'AssetCost':'mean', 
       'AmountFinance':'mean',
       'DisbursalAmount':'mean', 
       'ManufacturerID':'mean', 
       'LTV':'mean', 
       'SEX':'mean', 
       'AGE':'mean', 
       'MonthlyIncome':'mean', 
       'State':'mean',
      #  'Top-up Month':'mean', 
       'DURATION':'unique', 
       'lens':'mean', 
       'how_many_6':'mean',
       'REPORTED DATE - HISTm':'mean', 
       'REPORTED DATE - HISTy':'mean', 
       'DisbursalDatem':'mean',
       'DisbursalDatey':'mean', 
       'MaturityDAtem':'mean', 
       'MaturityDAtey':'mean',
    # 'DisbursalDatem':'mean',
    # 'DisbursalDatey':'mean',
    # 'MaturityDAtem':'mean',
    # 'MaturityDAtey':'mean',
    # 'EMI':'mean',
    # 'SELF-INDICATOR':'mean',
    # 'TENURE':'mean',
    # 'Frequency':'mean',
    # 'InstlmentMode':'mean',
    # 'LoanStatus':'sum',
    # 'PaymentMode':'mean',
    # 'Tenure':'mean',
    # 'AssetCost':'mean',
    # 'AmountFinance':'mean',
    # 'DisbursalAmount':'mean',
    # 'LTV':'mean','SEX':'mean',
    # 'AGE':'mean','MonthlyIncome':'mean',
    # 'DURATION':'unique',
    # 'lens':'mean'

}).reset_index()

In [ ]:
test.shape,train.shape

((14745, 33), (128655, 34))

In [ ]:
train['DURATION'] = train['DURATION'].apply(lambda x: x[0])
# train['Top-up Month'] = train['Top-up Month'].apply(lambda x: x[0])
test['DURATION'] = test['DURATION'].apply(lambda x: x[0])

In [ ]:
# traincur = traincur.fillna(0)
# testcur = testcur.fillna(0)

In [ ]:
train = train.merge(traincur,on='ID',how='left')
test = test.merge(testcur,on='ID',how='left')

In [ ]:
# train = train.fillna(0)
# test = test.fillna(0)

In [ ]:
def datestrp(x,s):
  x=str(x)
  if s=='m' and ('N' not in x):
    return x[5:7]
  else:
    return np.nan
  if s=='y' and ('N' not in x):
    return x[:4]
  else:
    return np.nan


In [ ]:
train['DURATION'] = train['DURATION'].apply(lambda x: float(str(x).split('days')[0]))
test['DURATION'] = test['DURATION'].apply(lambda x: float(str(x).split('days')[0]))

In [ ]:
# test.columns

Index([                   'ID',                   'EMI',
                   'ACCT-TYPE',      'CONTRIBUTOR-TYPE',
               'OWNERSHIP-IND',        'ACCOUNT-STATUS',
                 'ASSET_CLASS',        'SELF-INDICATOR',
                      'TENURE',             'Frequency',
               'InstlmentMode',            'LoanStatus',
                 'PaymentMode',              'BranchID',
                      'Tenure',             'AssetCost',
               'AmountFinance',       'DisbursalAmount',
              'ManufacturerID',                   'LTV',
                         'SEX',                   'AGE',
               'MonthlyIncome',                 'State',
                    'DURATION',                  'lens',
                  'how_many_6', 'REPORTED DATE - HISTm',
       'REPORTED DATE - HISTy',        'DisbursalDatem',
              'DisbursalDatey',         'MaturityDAtem',
               'MaturityDAtey',                     1.0,
                           2.0,

In [ ]:
# train = train.drop(columns= ['REPORTED DATE - HIST',
#                      'DisbursalDate',
#                      'MaturityDAte',
#                      'State',
#                      ])

# test = test.drop(columns= ['REPORTED DATE - HIST',
#                      'DisbursalDate',
#                      'MaturityDAte',
#                      'State',
#                      ])


In [ ]:
# train_x, test_x, train_y, test_y = train_test_split(trr.drop(columns=['ID','Top-up Month']), trr['Top-up Month'])

In [ ]:
# params = {'alpha': 0.00011211243738408213,
#  'booster': 'dart',
#  'eta': 0.016997707638062225,
#  'gamma': 4.3419043769837756e-08,
#  'grow_policy': 'depthwise',
#  'lambda': 0.007668374533807842,
#  'max_depth': 6,
#  'n_estimators': 291,
#  'normalize_type': 'tree',
#  'rate_drop': 7.535249046289386e-06,
#  'sample_type': 'uniform',
#  'skip_drop': 0.0035502603150640473}
# model = lightgbm.LGBMClassifier(n_jobs=-1)
                                
model = xgb.XGBClassifier(m_jobs=-1)
# from catboost import CatBoostClassifier
# model = catb.CatBoostClassifier(
#     # iterations=1000,
#     n_estimators=500
# #     verbose=5,
# )

# model.fit(
#     train_x, train_y,
#     cat_features=[],
#     eval_set=(test_x,test_y),
# )

In [ ]:
# wts = train_y.value_counts()/train.shape[0]
# wts = 0.006125/wts

In [ ]:
# wts

In [ ]:
# parameters = { #when use hyperthread, xgboost may become slower
              
#               'learning_rate': [0.05,0.1,0.001], #so called `eta` value
#               'max_depth': [6,7,8,10],
#               'min_child_weight': [11,50,100],
#               'subsample': [0.8],
#               'colsample_bytree': [0.7],
#               'n_estimators': [100,200,500], #number of trees, change it to 1000 for better results
#               'seed': [1337],
#               'reg_lambda':[0,1,10],
#               }


# clf = GridSearchCV(model, parameters, n_jobs=-1, 
#                    cv=StratifiedKFold(5), 
#                    verbose=3, refit=True)

# clf.fit(train.drop(columns=['ID','Top-up Month']), train["Top-up Month"])

In [ ]:

# clf = catb.CatBoostClassifier(
#     iterations=400,
# #     verbose=5,
# )

# clf.fit(
#     train_x, train_y,
#     cat_features=[],
#     eval_set=(test_x,test_y),
# )

In [ ]:
# # clf = model
# model.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, m_jobs=-1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
# # import shap
# explainer = shap.TreeExplainer(model)
# shap_values = explainer.shap_values(train_x.iloc[:2000,:])
# shap.summary_plot(shap_values,train_x.iloc[:2000,:] )

In [ ]:
# preds = model.predict(test_x)

# f1 = f1_score(test_y,preds,average='macro')

In [ ]:
# f1

0.38606935111063084

In [ ]:
# set(train.columns)-set(test.columns)

{'Top-up Month'}

In [ ]:
# preds = model.predict(tss.drop(columns=['ID']))

In [ ]:
# # preds = encode.inverse_transform(preds)
# trgt_inv_maps = {6:' > 48 Months', 
#              0:'No Top-up Service', 
#             1: '12-18 Months',
#               5:'36-48 Months', 
#               2:'18-24 Months', 
#               3:'24-30 Months', 
#               4:'30-36 Months'}
# preds = pd.Series(preds.reshape(-1,)).map(trgt_inv_maps)

In [ ]:
# test.head()

,ID,EMI,ACCT-TYPE,CONTRIBUTOR-TYPE,OWNERSHIP-IND,ACCOUNT-STATUS,ASSET_CLASS,SELF-INDICATOR,TENURE,Frequency,InstlmentMode,LoanStatus,PaymentMode,BranchID,Tenure,AssetCost,AmountFinance,DisbursalAmount,ManufacturerID,LTV,SEX,AGE,MonthlyIncome,State,DURATION,lens,how_many_6,REPORTED DATE - HISTm,REPORTED DATE - HISTy,DisbursalDatem,DisbursalDatey,MaturityDAtem,MaturityDAtey,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0
0,4,1000.0,1.314483,1.273236,1.377486,1.411054,1.135800,1,120.000000,2,0,95,0.664989,NaN,46,480000,365000.0,365000.0,0.708584,75.83,1,50.0,32069.00,0.874595,1376.0,95,20.210526,6.705263,2016.494751,12.0,2011.0,10.0,2015.0,0.000000,0.000000,0.000000,295675.333333,274852.333333,253139.333333,230498.000000,206888.333333,182269.000000,156596.666667,129826.666667,101911.666667,72804.000000,42451.666667,7182.666667,0.0,0.000000,0.000000,0.000000,0.0,0.0,1304707.0,1283732.000,1.256956e+06,1.235088e+06,1.213272e+06,1.191392e+06,1169797.000,1.148044e+06,1.125827e+06
1,5,9300.0,1.980461,1.348862,1.417879,1.464809,1.135800,1,180.000000,2,0,168,0.792426,NaN,45,480000,285000.0,285000.0,0.708584,57.44,1,35.0,25000.00,0.874595,1343.0,168,18.517857,6.577381,2016.416626,4.0,2012.0,1.0,2016.0,265770.666667,249787.666667,233145.666667,215734.333333,199323.666667,176564.666667,158975.666667,138411.333333,118472.000000,100733.666667,15504.000000,20672.000000,20672.000000,20672.000000,275000.000000,498666.0,252680.666667,171540.888889,145236.454545,151141.0,268967.5,294000.0,326002.625,4.320140e+05,4.283892e+05,3.377818e+05,3.140167e+05,166204.125,3.980911e+05,4.563858e+05
2,6,35800.0,1.374217,1.122616,1.354085,1.482727,1.190500,2,24.142857,3,1,161,0.792426,NaN,48,580000,400000.0,400000.0,0.715542,68.97,1,37.0,23333.33,0.874595,1419.0,161,10.838509,6.515528,2016.000000,10.0,2013.0,9.0,2017.0,0.000000,0.000000,0.000000,319454.000000,299245.000000,278050.000000,255820.000000,232505.000000,208052.000000,173440.000000,118222.800000,84184.500000,261548.454545,283076.666667,245978.000000,199335.0,197896.888889,170528.666667,96185.777778,155025.0,115650.0,96375.0,77100.000,8.673750e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000,0.000000e+00,0.000000e+00
3,25,52000.0,1.138935,1.463810,1.458272,1.437931,1.464001,1,NaN,1,1,70,0.696255,0.467202,36,725000,500000.0,500000.0,NaN,61.79,1,55.0,91666.67,0.486417,995.0,70,12.857143,6.585714,2016.085693,5.0,2013.0,2.0,2016.0,479066.666667,468600.000000,397260.000000,361590.000000,283552.666667,244534.000000,159171.333333,116490.000000,57113.333333,27425.000000,13712.500000,0.000000,0.000000,196696.000000,87880.000000,27399.0,21815.333333,32723.000000,10907.666667,0.0,98682.0,65768.0,50597.750,2.340900e+04,8.499067e+04,1.275930e+05,0.000000e+00,0.000,0.000000e+00,0.000000e+00
4,119,35400.0,1.079124,1.071563,1.349405,1.437931,1.135800,1,46.000000,3,1,72,0.792426,0.529412,48,617000,400000.0,400000.0,NaN,59.09,1,48.0,12500.00,0.861263,1466.0,72,17.000000,6.500000,2016.625000,11.0,2012.0,12.0,2016.0,0.000000,0.000000,0.000000,0.000000,0.000000,292696.500000,273979.666667,250601.666667,226198.000000,200723.666667,174131.333333,146372.000000,117394.333333,76772.333333,44741.000000,164778.0,293933.333333,408900.000000,497200.000000,450200.0,405200.0,360200.0,315200.000,2.722000e+05,2.332000e+05,1.942000e+05,1.552000e+05,122700.000,0.000000e+00,0.000000e+00


In [ ]:
# preds_csv = pd.read_csv('/content/drive/MyDrive/sample_submission_ejm25Dc.csv')

In [ ]:
# preds_csv.head()

,ID,Top-up Month
0,4,36-48 Months
1,5,18-24 Months
2,6,30-36 Months
3,25,30-36 Months
4,119,30-36 Months


In [ ]:
# len(preds),len(test)

(14745, 14745)

In [ ]:
# preds_csv['Top-up Month'] = preds

In [ ]:
# preds_csv['Top-up Month'].value_counts()

No Top-up Service    14315
 > 48 Months           346
24-30 Months            42
30-36 Months            18
18-24 Months            15
36-48 Months             5
12-18 Months             4
Name: Top-up Month, dtype: int64

In [ ]:
# preds_csv.to_csv('preds1.csv',index = None)

In [ ]:
# preds_csv['ID'] == test['ID']

0        True
1        True
2        True
3        True
4        True
         ... 
14740    True
14741    True
14742    True
14743    True
14744    True
Name: ID, Length: 14745, dtype: bool

In [ ]:
# preds_csv.to_csv('/content/drive/MyDrive/preds1.csv',index = None)

# preds_csv['Top-up Month'].value_counts()

No Top-up Service    14286
 > 48 Months           389
24-30 Months            26
18-24 Months            16
30-36 Months            15
36-48 Months            10
12-18 Months             3
Name: Top-up Month, dtype: int64

In [ ]:
train.to_csv('/content/drive/MyDrive/trainer_csv1.csv',index=None)
test.to_csv('/content/drive/MyDrive/tester_csv1.csv',index=None)

In [ ]:
# !pip install optuna

In [ ]:
# import optuna

In [ ]:
# def objective(trial):
#     # data, target = train.drop(columns=['ID','Top-up Month']), train['Top-up Month']
#     # train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
#     # dtrain = xgb.DMatrix(train_x, label=train_y)
#     # dtest = xgb.DMatrix(test_x, label=test_y)

#     param = {
#         # "silent": 1,
#         "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
#         "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
#         "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
#     }

#     if param["booster"] == "gbtree" or param["booster"] == "dart":
#         param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
#         param["n_estimators"] = trial.suggest_int("n_estimators", 100, 300)
#         param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
#         param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
#         param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
#     if param["booster"] == "dart":
#         param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
#         param["n_estimators"] = trial.suggest_int("n_estimators", 100, 300)
#         param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
#         param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
#         param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)

#     # Add a callback for pruning.
#     # pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
#     bst = lightgbm.LGBMClassifier(n_jobs=-1,**param)
#     bst.fit(train_x,train_y)
#     preds = bst.predict(test_x)
#     accuracy = f1_score(test_y, preds,average='macro')
#     return accuracy

In [ ]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=20)
# print(study.best_trial)

[I 2021-02-06 16:32:38,276] A new study created in memory with name: no-name-a8797296-678e-453b-a5dd-457f9179c42c
[I 2021-02-06 16:33:14,292] Trial 0 finished with value: 0.46307102336329997 and parameters: {'booster': 'gblinear', 'lambda': 2.0023502339105562e-05, 'alpha': 0.5711689972067657}. Best is trial 0 with value: 0.46307102336329997.
[I 2021-02-06 16:34:41,230] Trial 1 finished with value: 0.4709284649097033 and parameters: {'booster': 'dart', 'lambda': 0.18233922124833515, 'alpha': 0.0036469510475322623, 'max_depth': 7, 'n_estimators': 287, 'eta': 0.0216747663312638, 'gamma': 2.201317684833781e-08, 'grow_policy': 'depthwise', 'sample_type': 'weighted', 'normalize_type': 'tree', 'rate_drop': 4.7213489679524155e-07, 'skip_drop': 7.970392615589048e-08}. Best is trial 1 with value: 0.4709284649097033.
[I 2021-02-06 16:35:05,057] Trial 2 finished with value: 0.378884552988136 and parameters: {'booster': 'dart', 'lambda': 3.4794104401237447e-07, 'alpha': 7.219851127503527e-06, 'max_

FrozenTrial(number=13, values=[0.47408314357303366], datetime_start=datetime.datetime(2021, 2, 6, 16, 42, 52, 928408), datetime_complete=datetime.datetime(2021, 2, 6, 16, 44, 13, 161220), params={'booster': 'dart', 'lambda': 0.08321014907053136, 'alpha': 0.6975157004706682, 'max_depth': 6, 'n_estimators': 299, 'eta': 0.5402652055257229, 'gamma': 1.2195761205326334e-08, 'grow_policy': 'lossguide', 'sample_type': 'weighted', 'normalize_type': 'forest', 'rate_drop': 2.0500382769560446e-08, 'skip_drop': 1.198304390656999e-08}, distributions={'booster': CategoricalDistribution(choices=('gbtree', 'gblinear', 'dart')), 'lambda': LogUniformDistribution(high=1.0, low=1e-08), 'alpha': LogUniformDistribution(high=1.0, low=1e-08), 'max_depth': IntUniformDistribution(high=9, low=1, step=1), 'n_estimators': IntUniformDistribution(high=300, low=100, step=1), 'eta': LogUniformDistribution(high=1.0, low=1e-08), 'gamma': LogUniformDistribution(high=1.0, low=1e-08), 'grow_policy': CategoricalDistribution